# **Carregamento dos dados**

In [5]:
!pip install sqlalchemy pymysql
from sqlalchemy import create_engine
import numpy as np
import pandas as pd

# Definir as credenciais do RDS
endpoint = 'db-hotel.c1yumw4g6gf8.us-east-1.rds.amazonaws.com'
port = 3306
username = 'admin'
password = 'empRgyRBCEE5I56SwgJ1'
dbname = 'hotel_reservations'


In [ ]:
#Define a conexão com o rds
engine = create_engine(f'mysql+pymysql://{username}:{password}@{endpoint}:{port}/{dbname}')

# Consultar os dados de uma tabela
query = "SELECT * FROM reservation_Updated"
base_hotel = pd.read_sql(query, engine)
base_hotel


In [7]:
#encerra a conexão
engine.dispose()

In [8]:
base_hotel


,label_avg_price_per_room,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,...,room_type_reserved_Room_Type_5,room_type_reserved_Room_Type_6,room_type_reserved_Room_Type_7,market_segment_type_Aviation,market_segment_type_Complementary,market_segment_type_Corporate,market_segment_type_Offline,market_segment_type_Online,booking_status_Canceled,booking_status_Not_Canceled
0,1,2,0,1,2,0,224,2017,10,2,...,0,0,0,0,0,0,1,0,0,1
1,2,2,0,2,3,0,5,2018,11,6,...,0,0,0,0,0,0,0,1,0,1
2,1,1,0,2,1,0,1,2018,2,28,...,0,0,0,0,0,0,0,1,1,0
3,2,2,0,0,2,0,211,2018,5,20,...,0,0,0,0,0,0,0,1,1,0
4,2,2,0,1,1,0,48,2018,4,11,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,3,3,0,2,6,0,85,2018,8,3,...,0,0,0,0,0,0,0,1,0,1
36271,2,2,0,1,3,0,228,2018,10,17,...,0,0,0,0,0,0,0,1,1,0
36272,2,2,0,2,6,0,148,2018,7,1,...,0,0,0,0,0,0,0,1,0,1
36273,2,2,0,0,3,0,63,2018,4,21,...,0,0,0,0,0,0,0,1,1,0


# **Organizando as bases para treinamento e teste**

In [9]:
#remapeando a coluna alvo para 0, 1 e 2 para ser compatível com o modelo de classificação
def remapear(categoria):
    if categoria == 1:
        return 0
    elif categoria  == 2:
        return 1
    else:
        return 2


base_hotel['label_avg_price_per_room'] = base_hotel['label_avg_price_per_room'].apply(remapear)
base_hotel


,label_avg_price_per_room,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,...,room_type_reserved_Room_Type_5,room_type_reserved_Room_Type_6,room_type_reserved_Room_Type_7,market_segment_type_Aviation,market_segment_type_Complementary,market_segment_type_Corporate,market_segment_type_Offline,market_segment_type_Online,booking_status_Canceled,booking_status_Not_Canceled
0,0,2,0,1,2,0,224,2017,10,2,...,0,0,0,0,0,0,1,0,0,1
1,1,2,0,2,3,0,5,2018,11,6,...,0,0,0,0,0,0,0,1,0,1
2,0,1,0,2,1,0,1,2018,2,28,...,0,0,0,0,0,0,0,1,1,0
3,1,2,0,0,2,0,211,2018,5,20,...,0,0,0,0,0,0,0,1,1,0
4,1,2,0,1,1,0,48,2018,4,11,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,2,3,0,2,6,0,85,2018,8,3,...,0,0,0,0,0,0,0,1,0,1
36271,1,2,0,1,3,0,228,2018,10,17,...,0,0,0,0,0,0,0,1,1,0
36272,1,2,0,2,6,0,148,2018,7,1,...,0,0,0,0,0,0,0,1,0,1
36273,1,2,0,0,3,0,63,2018,4,21,...,0,0,0,0,0,0,0,1,1,0


In [ ]:
base_treinamento = base_hotel.iloc[0:22792,:]
base_treinamento.shape

(22792, 32)

In [ ]:
base_teste = base_hotel.iloc[22792:,:]
base_teste.shape

(13483, 32)

In [ ]:
22792 + 13483


36275

In [ ]:
X_teste = base_teste.iloc[:,1:len(base_hotel)].values
X_teste

array([[3, 0, 5, ..., 1, 1, 0],
       [1, 0, 1, ..., 1, 0, 1],
       [2, 0, 1, ..., 0, 1, 0],
       ...,
       [2, 0, 2, ..., 1, 0, 1],
       [2, 0, 0, ..., 1, 1, 0],
       [2, 0, 1, ..., 0, 0, 1]])

In [ ]:
X_teste.shape

(13483, 31)

In [ ]:
y_teste = base_teste.iloc[:, 0].values
y_teste.shape

(13483,)

In [ ]:
base_treinamento.to_csv('hotel_train_xgboost.csv', header = False, index = False)
base_teste.to_csv('hotel_test_xgboost.csv', header = False, index = False)

# **Configuração do Sagemaker**

In [ ]:
import sagemaker
import boto3
from sagemaker import Session
import sagemaker.amazon.common as smac
import io
import os

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [ ]:
session = sagemaker.Session()
bucket = 'testeprojetohotel'
subpasta_modelo = 'modelos/hotel/xgboost'
subpasta_dataset = 'datasets/hotel'
key_train = 'hotel-train-data-xgboost'
key_test = 'hotel-test-data-xgboost'
role = sagemaker.get_execution_role()
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key_train)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, subpasta_dataset, key_test)
output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo)
print('Role: {}'.format(role))
print('Localização da base de treinamento: {}'.format(s3_train_data))
print('Localização da base de teste: {}'.format(s3_test_data))
print('Modelo final será salvo em: {}'.format(output_location))

Role: arn:aws:iam::471112624624:role/service-role/AmazonSageMaker-ExecutionRole-20240913T094282
Localização da base de treinamento: s3://testeprojetohotel/datasets/hotel/train/hotel-train-data-xgboost
Localização da base de teste: s3://testeprojetohotel/datasets/hotel/test/hotel-test-data-xgboost
Modelo final será salvo em: s3://testeprojetohotel/modelos/hotel/xgboost/output


In [ ]:
with open('hotel_train_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key_train)).upload_fileobj(f)

In [ ]:
with open('hotel_test_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'test', key_test)).upload_fileobj(f)

# **Treinamento do Modelo no Sagemaker**

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/ecr-sa-east-1.html
from sagemaker import image_uris
container = image_uris.retrieve(framework = 'xgboost', region=boto3.Session().region_name, version='latest')

In [ ]:
# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
#
xgboost = sagemaker.estimator.Estimator(image_uri = container,
                                        role = role,
                                        instance_count = 1,
                                        instance_type = 'ml.m5.2xlarge',
                                        output_path = output_location,
                                        sagemaker_session = session)

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
xgboost.set_hyperparameters(num_round = 100, objective = 'multi:softmax', num_class = 3)

In [ ]:
train_input = sagemaker.inputs.TrainingInput(s3_data = s3_train_data, content_type='csv', s3_data_type = 'S3Prefix')
validation_input = sagemaker.inputs.TrainingInput(s3_data = s3_test_data, content_type='csv', s3_data_type = 'S3Prefix')
data_channels = {'train': train_input, 'validation': validation_input}

In [ ]:
xgboost.fit(data_channels)

INFO:sagemaker:Creating training-job with name: xgboost-2024-09-13-16-09-04-431


2024-09-13 16:09:04 Starting - Starting the training job...
2024-09-13 16:09:31 Starting - Preparing the instances for training...
2024-09-13 16:10:03 Downloading - Downloading input data...
2024-09-13 16:10:18 Downloading - Downloading the training image...
2024-09-13 16:10:59 Training - Training image download completed. Training in progress...Arguments: train
[2024-09-13:16:11:17:INFO] Running standalone xgboost training.
[2024-09-13:16:11:17:INFO] File size need to be processed in the node: 2.39mb. Available memory size in the node: 23534.86mb
[2024-09-13:16:11:17:INFO] Determined delimiter of CSV input is ','
[16:11:17] S3DistributionType set as FullyReplicated
[16:11:17] 22792x31 matrix with 706552 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-09-13:16:11:17:INFO] Determined delimiter of CSV input is ','
[16:11:17] S3DistributionType set as FullyReplicated
[16:11:17] 13483x31 matrix with 417973 entries loaded from /opt/ml/input/data/vali

# Deploy, previsões e avaliação

In [ ]:
xgboost_classifier = xgboost.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2024-09-13-16-11-51-650
INFO:sagemaker:Creating endpoint-config with name xgboost-2024-09-13-16-11-51-650
INFO:sagemaker:Creating endpoint with name xgboost-2024-09-13-16-11-51-650


--------!

In [ ]:
from sagemaker.serializers import CSVSerializer
xgboost_classifier.serializer = CSVSerializer()

In [ ]:
X_teste.shape, type(X_teste)

((13483, 31), numpy.ndarray)

In [ ]:
previsoes = np.array(xgboost_classifier.predict(X_teste).decode('utf-8').split(',')).astype(np.float32)
previsoes

array([2., 1., 0., ..., 1., 1., 2.], dtype=float32)

In [ ]:

print(previsoes)

[2. 1. 0. ... 1. 1. 2.]


In [ ]:
previsoes.shape, y_teste.shape

((13483,), (13483,))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
cm = confusion_matrix(y_teste, previsoes)
cm

array([[3427,  599,   92],
       [ 486, 4042,  497],
       [  62,  490, 3788]])

In [ ]:
accuracy_score(y_teste, previsoes)

0.8349032114514574

In [ ]:
print(classification_report(y_teste, previsoes))

              precision    recall  f1-score   support

           0       0.86      0.83      0.85      4118
           1       0.79      0.80      0.80      5025
           2       0.87      0.87      0.87      4340

    accuracy                           0.83     13483
   macro avg       0.84      0.84      0.84     13483
weighted avg       0.84      0.83      0.84     13483



# **Tuning**